# Import packages

In [20]:
import re
from textblob import TextBlob
import numpy as np
np.set_printoptions(linewidth=110)
import pandas as pd 
import random
import string
from tqdm import tqdm, tqdm_notebook
import os
import json
import sys
import spacy
import scispacy  # https://allenai.github.io/scispacy/
spacy.prefer_gpu()
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
sys.path.insert(0, "../")
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)
import uuid

from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load data from CDC and start processing for NER

In [13]:
df=pd.read_csv('D:/GitHub/COVID-19/data/CDC/All_Articles_Excel_trim15.csv',encoding='ISO-8859-1', usecols=["Abstract","Title","Author","Journal","Year","Name of Database"])

In [3]:
df=pd.read_csv('D:/GitHub/COVID-19/data/CDC/All_Articles_Excel_trim3.csv',encoding='ISO-8859-1', usecols=["Abstract","Title"])

In [4]:
#Count the articls we have
df.shape

(4, 2)

In [5]:
# remove articles that do no have an Abstract since that is what we are using for NER
df = df[df['Abstract'].notna()]
df.shape

(3, 2)

In [14]:
df.head()

,Author,Title,Abstract,Year,Journal,Name of Database
0,ç¨ç,COVID-19æ¯ä¸é»é©å½ç æ¯ãèå»¶æ¼åç·...,NaN,2020,æµ·å³½è©è«,World Health Organization
1,"Zuoru, LM, Pang; Xinping, Yang; Jie, Li; Yufei...",Weekly Assessment of the COVID-19 Pandemic and...,NaN,2020,China CDC Weekly,China CDC Weekly
2,"Zuckier, LSG, Sharon R.",Pain Management Under COVID-19 COVID-19 in the...,NaN,2020,Topics in Pain Management,World Health Organization
3,"Zong, YD, M. L.; Ma, S. T.; Ju, W. Z.",Investigation of potential Chinese materia med...,Objective: To investigate the mechanism of tre...,2020,Chinese Traditional and Herbal Drugs,World Health Organization
4,"Zimmermann, GW",Coronavirus: Protective clothing will be centr...,NaN,2020,MMW-Fortschritte der Medizin,Scopus


In [7]:
df['uuid'] = df.apply(lambda x: uuid.uuid1(), axis=1)

In [8]:
df.head()

,Title,Abstract,uuid
0,Preliminary study of traditional Chinese medic...,this is an abstract talking about COVID-19,4f37931a-8bcb-11ea-970d-104a7d4533bd
2,CT scans of patients with 2019 novel coronavir...,here is another abstract COVID-19 and the ACE2...,4f37ba18-8bcb-11ea-8bab-104a7d4533bd
3,Geographical Distance to the Epicenter of Covi...,"finally, the last abstract talking about liver...",4f37ba19-8bcb-11ea-bc0b-104a7d4533bd


## Visualize Bio-Medical entities 

In [9]:
# This function will allow us to randomly color the bio-medical entities that teh spacy NER model finds.
# THis is mainly for display purposes
import random 

def get_entity_options(random_colors=False):
    """
    generating color options for visualizing the named entities
    """
    def color_generator(number_of_colors):
        color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
        return color

    entities = ["GGP", "SO", "TAXON", "CHEBI", "GO", "CL", 
                "DNA", "CELL_TYPE", "CELL_LINE", "RNA", "PROTEIN",
                "DISEASE", "CHEMICAL",
                "CANCER", "ORGAN", "TISSUE", "ORGANISM", "CELL", 
                "AMINO_ACID", "GENE_OR_GENE_PRODUCT", "SIMPLE_CHEMICAL", 
                "ANATOMICAL_SYSTEM", "IMMATERIAL_ANATOMICAL_ENTITY", "MULTI-TISSUE_STRUCTURE",
                "DEVELOPING_ANATOMICAL_STRUCTURE", "ORGANISM_SUBDIVISION", "CELLULAR_COMPONENT"]
    
    colors = {"ENT":"#82E0AA"}
    
    if random_colors:
        color = color_generator(len(entities))
        for i in range(len(entities)):
            colors[entities[i]] = color[i]
    else:
        entities_cat_1 = {"GGP":"#F9E79F", "SO":"#F7DC6F", "TAXON":"#F4D03F", "CHEBI":"#FAD7A0", "GO":"#F8C471", "CL":"#F5B041"}
        entities_cat_2 = {"DNA":"#82E0AA", "CELL_TYPE":"#AED6F1", "CELL_LINE":"#E8DAEF", "RNA":"#82E0AA", "PROTEIN":"#82E0AA"}
        entities_cat_3 = {"DISEASE":"#D7BDE2", "CHEMICAL":"#D2B4DE"}
        entities_cat_4 = {"CANCER":"#ABEBC6", "ORGAN":"#82E0AA", "TISSUE":"#A9DFBF", "ORGANISM":"#A2D9CE", "CELL":"#76D7C4", "AMINO_ACID":"#85C1E9", "GENE_OR_GENE_PRODUCT":"#AED6F1", "SIMPLE_CHEMICAL":"#76D7C4", "ANATOMICAL_SYSTEM":"#82E0AA", "IMMATERIAL_ANATOMICAL_ENTITY":"#A2D9CE", "MULTI-TISSUE_STRUCTURE":"#85C1E9", "DEVELOPING_ANATOMICAL_STRUCTURE":"#A9DFBF", "ORGANISM_SUBDIVISION":"#58D68D", "CELLULAR_COMPONENT":"#7FB3D5"}

        entities_cats = [entities_cat_1, entities_cat_2, entities_cat_3, entities_cat_4]
        for item in entities_cats:
            colors = {**colors, **item}
    
    options = {"ents": entities, "colors": colors}
    
    return options

In [15]:
# Text block for NER testing
text_test = str("Angiotensin-converting enzyme 2 ( ACE2 ) as aSARS-CoV-2 receptor: molecular mechanisms and potential therapeutic target.SARS-CoV-2 has been sequenced [ 3  ] . A phylogenetic’ analysis [ 3  , 4  ] found a bat origin for the SARS-CoV-2  . There is a diversity of possible intermediate hosts for SARS-CoV-2  , including pangolins  , but not Mice and rats) | 5  | . There are many similarities of SARS-CoV-2 with the original SARS-CoV  . Using computer modeling , Xu et al . [ 6 |] found that the spike proteins  of SARS-CoV-2 and SARS-CoV  have almost identical 3-D structures in thereceptor binding domain that maintains Van der Waals forces . SARS-CoV spike proteins  has a strong binding affinity to human ACE2 , based on biochemical interaction studies and crystal structure analysis [7  ] . SARS-CoV-2  and SARS-CoV spike proteins share identity in amino acid sequences and ...... ")

In [19]:
from spacy import displacy
nlp = spacy.load("en_ner_bionlp13cg_md")
doc = nlp(text_test)
displacy.render(doc, style="ent", options=get_entity_options(random_colors=True))

## Lets apply the NER model to the Abstract's and output that to a dataframe that is ready for import into Neo4j as node's

In [36]:
getattr(tqdm, '_instances', {}).clear()  # ⬅ clear the progress
spacy.prefer_gpu()
entities=[]
for i in tqdm(df['Abstract'].tolist()):
    doc = nlp(i)
    for entity in doc.ents:
        entities.append((i, entity.text, entity.label_, df.uuid))

100%|████████████████████████████████████████████████████| 3/3 [00:00<00:00, 103.41it/s]


In [37]:
print(entities)

[('this is an abstract talking about COVID-19', 'COVID-19', 'SIMPLE_CHEMICAL', 0    e7fc8254-8645-11ea-8099-104a7d4533bd
2    e7fc8255-8645-11ea-ac58-104a7d4533bd
3    e7fc8256-8645-11ea-85f5-104a7d4533bd
Name: uuid, dtype: object), ('here is another abstract COVID-19 and the ACE2 gene', 'COVID-19', 'GENE_OR_GENE_PRODUCT', 0    e7fc8254-8645-11ea-8099-104a7d4533bd
2    e7fc8255-8645-11ea-ac58-104a7d4533bd
3    e7fc8256-8645-11ea-85f5-104a7d4533bd
Name: uuid, dtype: object), ('here is another abstract COVID-19 and the ACE2 gene', 'ACE2', 'GENE_OR_GENE_PRODUCT', 0    e7fc8254-8645-11ea-8099-104a7d4533bd
2    e7fc8255-8645-11ea-ac58-104a7d4533bd
3    e7fc8256-8645-11ea-85f5-104a7d4533bd
Name: uuid, dtype: object), ('finally, the last abstract talking about liver, lung, ACE2, heart', 'liver', 'ORGAN', 0    e7fc8254-8645-11ea-8099-104a7d4533bd
2    e7fc8255-8645-11ea-ac58-104a7d4533bd
3    e7fc8256-8645-11ea-85f5-104a7d4533bd
Name: uuid, dtype: object), ('finally, the last abstract talking 

In [38]:
df_ner = pd.DataFrame.from_records(entities, columns =["Abstract",'ent_name', 'ent_type','uuid'])

In [39]:
df_ner.to_csv("ner_out_uuid.csv", sep=',',index=False)

## ToDo

1. Add UUID to all abstracts-DONE
2. Add UUID to entity output for matching later
3. Filter entities so if it appears more than once it only shows up in output
4. For filtered entities, keep a count of the number of times it was mentioned
5. Extract the authors individually 
5a. In some Journals, the author named first is primary but in other journals it is the last author
6. See if we can find if that author is cited in the article